In [1]:
import netCDF4 
import pandas as pd
import os.path
import numpy as np
from datetime import datetime, timedelta
from cftime import num2date, date2num

dataPath = "./raw_data/"
# dataPath = "./raw_data/Maebong_and_Moonam/"
 
files = os.listdir(dataPath)

datasetDict = {}
for idx,nc_type_file in enumerate(files):
    if nc_type_file.split(".")[-1] == 'nc':
        datasetDict[nc_type_file] = netCDF4.Dataset(dataPath+nc_type_file,mode="r")

In [2]:
# 하단에 저장되어 있습니다.
datasetDict.keys()

dict_keys(['20201114_testdata.nc', '20201114_testdata_valid.nc', '20210731_mundo.nc', '20210731_wando.nc', '20210816_wando.nc'])

In [18]:
ncfile_name = '20201114_testdata.nc'
output_dataset = datasetDict[ncfile_name]

# 하단 데이터를 갖고 있습니다.
output_dataset.variables.keys()

#mdww : mean direction of wind waves
#mpww : mean period of wind waves
#mwd : mean wave direction
#mwp : mean wave period
#wmb : Model bathymetry
#dwi : Ocean surface stress equivalent 10m neutral wind direction
#swh : Significant height of combined wind waves and swell
#shww : significant height of wind waves

dict_keys(['longitude', 'latitude', 'time', 'u10', 'v10', 'mdww', 'mpww', 'mwd', 'mwp', 'wmb', 'dwi', 'swh', 'shww'])

In [5]:
output_dataset['time']

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:00.0
    long_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (8784,)
filling on, default _FillValue of -2147483647 used

In [50]:
output_dataset['longitude']

output_dataset['time'].get_dims()[0].size

<class 'netCDF4._netCDF4.Variable'>
float32 longitude(longitude)
    units: degrees_east
    long_name: longitude
unlimited dimensions: 
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [21]:
# t2m = output_dataset.variables
time_dim, lat_dim, lon_dim = output_dataset['time'].get_dims()

ValueError: not enough values to unpack (expected 3, got 1)

In [20]:
output_dataset['u10'].get_dims()

(<class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 8760,
 <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 1,
 <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 1)

In [27]:
# output_dataset.variables

In [29]:
output_dataset.variables['time']

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:00.0
    long_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (8760,)
filling on, default _FillValue of -2147483647 used

In [30]:
output_dataset['time']

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:00.0
    long_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (8760,)
filling on, default _FillValue of -2147483647 used

In [45]:
t2m

NameError: name 't2m' is not defined

In [54]:
output_dataset.variables['expver']

<class 'netCDF4._netCDF4.Variable'>
int32 expver(expver)
    long_name: expver
unlimited dimensions: 
current shape = (2,)
filling on, default _FillValue of -2147483647 used

In [78]:
timedim = output_dataset['time'].get_dims()[0].size

time_var = output_dataset.variables[time_dim.name]
times = num2date(time_var[:], time_var.units)


AttributeError: 'int' object has no attribute 'name'

In [32]:
time_var = output_dataset.variables['time']
times = num2date(time_var[:], time_var.units)


In [34]:
str(times[0])

'2019-01-01 00:00:00'

In [35]:
time_var.units

'hours since 1900-01-01 00:00:00.0'

In [36]:
meta_data

NameError: name 'meta_data' is not defined

In [4]:
output_dataset = datasetDict['20210731_wando.nc']
output_dataset.variables.keys()

dict_keys(['longitude', 'latitude', 'time', 'u100', 'v100'])

In [38]:
target_dataset = '20201114_testdata.nc'
output_dataset = datasetDict[target_dataset]

output_dataset.variables.keys()

dict_keys(['longitude', 'latitude', 'time', 'u10', 'v10', 'mdww', 'mpww', 'mwd', 'mwp', 'wmb', 'dwi', 'swh', 'shww'])

In [40]:
outputDict = {}

# output_dataset = datasetDict['20210731_mundo.nc']
output_list = []
meta_data = []
column_name = []

for key_value in ["latitude","longitude"]:
    meta_data.append(output_dataset.variables[key_value][:].data)

    
for key_value in ['time']:
#     output_dataset['time'].get_dims()
    dtime = netCDF4.num2date(output_dataset['time'],output_dataset['time'].units)
    output_list.append(np.array([dtime.data]).T.reshape([8760 ,1,1]))
    column_name.append(key_value)

    
for key_value in output_dataset.variables.keys():
    if key_value not in ["time","latitude","longitude",'expver']:
        output_list.append(output_dataset.variables[key_value][:].data)
        column_name.append(key_value)

meta_data

[array([35.], dtype=float32), array([125.75], dtype=float32)]

In [43]:
len(output_list)

11

In [159]:
output_dataset['time']

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:00.0
    long_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (8760,)
filling on, default _FillValue of -2147483647 used

In [44]:
times = [str(i) for i in netCDF4.num2date(output_dataset['time'],output_dataset['time'].units)]

times = np.array(times)

len(times)

8760

In [46]:
times =  times.reshape(len(times),1)
aims = []
aims.append(times)

for i in output_list[1:]:
    aims.append(i[:,0,0].reshape(len(i[:,0,0]),1))

In [47]:
output_df = pd.DataFrame(np.concatenate(aims,axis=1), columns =column_name)

# output_df = pd.DataFrame(np.concatenate(output_list,axis=1).reshape(17520,5),columns=column_name)
output_df.head()

,time,u10,v10,mdww,mpww,mwd,mwp,wmb,dwi,swh,shww
0,2019-01-01 00:00:00,5.112340373689354,-6.201502070743386,321.44740323015253,4.3147192580244464,323.6714227484878,4.57838306968649,32.0,318.71212033988354,1.1893830037922126,1.0545758876672768
1,2019-01-01 01:00:00,5.390213602249491,-6.026636362988979,319.04699799454295,4.398470999212869,321.3149844651281,4.6139138003757605,32.0,315.6745056612739,1.184784548831765,1.0770551561809025
2,2019-01-01 02:00:00,6.5670571993062845,-6.441110445382643,316.8003944673935,4.56278393906825,318.6509458464113,4.626944630701899,32.0,313.2850619267436,1.2526259522605914,1.1995442315510656
3,2019-01-01 03:00:00,7.125460188248357,-6.7835810010539035,315.613924144987,4.6608931216032605,316.60210789840846,4.687668300021703,32.0,312.977455377011,1.3696310840319814,1.3304056875410994
4,2019-01-01 04:00:00,7.424054365018523,-7.051022671737114,315.15251901960664,4.78107022324506,315.67930689233754,4.803034584509113,32.0,313.3784424864839,1.4798236899360417,1.4424579596727938


In [50]:
from metpy.calc import wind_speed, wind_direction
from metpy.units import units

def recordFunction(record):
    u = float(record['u10']) * units.meter / units.second
    v = float(record['v10']) * units.meter / units.second
    return [wind_direction(u,v), wind_speed(u,v)]

In [51]:
listofTwo = output_df.apply(recordFunction, axis=1)

# type(listofTwo.tolist()[0][0])

# listofTwo.tolist()[0][0].to_compact

a = pd.DataFrame(listofTwo.tolist(),columns =['direction','wind_speed'])

column_name.append('wind_direction')
column_name.append('wind_speed')
column_name

['time',
 'u10',
 'v10',
 'mdww',
 'mpww',
 'mwd',
 'mwp',
 'wmb',
 'dwi',
 'swh',
 'shww',
 'wind_direction',
 'wind_speed']

In [52]:
output_df1 = pd.DataFrame(np.concatenate([output_df,a],axis=1),columns = column_name)

file_name = str(meta_data[0][0])+"(lat) "+str(meta_data[1][0])+"(lon)1 located data"+'.csv'
output_df1.to_csv(file_name,index=True, header=True)

In [53]:
file_name

'35.0(lat) 125.75(lon)1 located data.csv'

In [105]:
 a = pd.DataFrame(listofTwo.tolist(),columns =['direction','wind_speed'])

In [35]:
file_name = str(meta_data[0][0])+"(lat) "+str(meta_data[1][0])+"(lon)1 located data"+'.csv'
# output_df.to_csv(file_name,index=True, header=True)

In [36]:
output_df1.to_csv(file_name,index=True, header=True)

In [128]:
pd.concat([output_df,a],axis=1).to_csv(file_name,index=True, header=True)

In [61]:
output_df.shape

(17520, 5)

In [42]:
output_dataset['time'].units

C:\Users\pc\.conda\envs\era05_setup_37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  """Entry point for launching an IPython kernel.


'hours since 1900-01-01 00:00:00.0'

In [79]:
meta_data[0][0]

36.44

In [80]:
meta_data[1][0]

128.42